In [5]:
#########Initialize##########
import re
import json
import time
import math
import requests
import pyperclip as pc
from decimal import Decimal
from web3 import Web3
from ens import ENS
from ipywidgets import widgets, Layout
from IPython.display import display, Javascript

###########NODE INFO##############
web3 = Web3(Web3.HTTPProvider("https://mainnet.infura.io/v3/bdfb19d0e7914c1c975dd89500550e06"))
######################################

ns=ENS.fromWeb3(web3)

#CALLS MUST BE SUCCESFFUL
sushiswapQuoterContract=Web3.toChecksumAddress("0xd9e1cE17f2641f24aE83637ab66a2cca9C378B9F")
sushiswapFactoryContract=Web3.toChecksumAddress("0xC0AEe478e3658e2610c5F7A4A2E1777cE9e4f2Ac")
WETH_address=Web3.toChecksumAddress("0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2")
ALCX_address=Web3.toChecksumAddress("0xdBdb4d16EdA451D0503b854CF79D55697F90c8DF")
DAI_address=Web3.toChecksumAddress("0x6B175474E89094C44Da98b954EedeAC495271d0F")
CVX_address=Web3.toChecksumAddress("0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B")
ALCXETH_pool=Web3.toChecksumAddress("0xC3f279090a47e80990Fe3a9c30d24Cb117EF91a8")

tokenDict={'WETH': WETH_address,'DAI': DAI_address, 'CVX': CVX_address, 'ALCX-ETH': ALCXETH_pool}

bondContract_WETHALCX_address=Web3.toChecksumAddress("0x6D987Dda4358a9Ba4F695Ca4e6AA3933D7Bc1B08")
bondContract_DAIALCX_address=Web3.toChecksumAddress("0x105cD312418311dd49Bc1e88459019Ed4eF218DC")
bondContract_CVXALCX_address=Web3.toChecksumAddress("0xd9feE3CdbF0B55461977756dBAEE4421Cc0A52E0")
bondContract_ALCXETHSLP_address=Web3.toChecksumAddress("0x1e5b7412f4B4B713b93D0e82260BD27810984B6e")

reedeemSwapLPBondABI='[ { "inputs": [], "stateMutability": "nonpayable", "type": "constructor" }, { "inputs": [], "name": "oneInchRouter", "outputs": [ { "internalType": "address", "name": "", "type": "address" } ], "stateMutability": "view", "type": "function" }, { "inputs": [], "name": "owner", "outputs": [ { "internalType": "address", "name": "", "type": "address" } ], "stateMutability": "view", "type": "function" }, { "inputs": [ { "components": [ { "internalType": "address[]", "name": "redeems", "type": "address[]" }, { "internalType": "uint256[4]", "name": "slippages", "type": "uint256[4]" }, { "internalType": "uint256", "name": "amountIn", "type": "uint256" }, { "internalType": "address", "name": "tokenIn", "type": "address" }, { "internalType": "address", "name": "tokenOut", "type": "address" }, { "internalType": "bytes", "name": "swapPayload", "type": "bytes" }, { "internalType": "bool", "name": "LP", "type": "bool" }, { "internalType": "address", "name": "poolAddress", "type": "address" }, { "internalType": "address", "name": "bondContractAddress", "type": "address" } ], "internalType": "struct RedeemInvest.Params", "name": "_params", "type": "tuple" } ], "name": "redeemSwapLPBond", "outputs": [], "stateMutability": "nonpayable", "type": "function" }, { "inputs": [ { "internalType": "address", "name": "_tokenContract", "type": "address" }, { "internalType": "uint256", "name": "_amount", "type": "uint256" } ], "name": "saveTokens", "outputs": [], "stateMutability": "nonpayable", "type": "function" }, { "inputs": [ { "internalType": "address", "name": "newOwner", "type": "address" } ], "name": "setOwner", "outputs": [], "stateMutability": "nonpayable", "type": "function" }, { "inputs": [], "name": "sushiRouter", "outputs": [ { "internalType": "address", "name": "", "type": "address" } ], "stateMutability": "view", "type": "function" } ]'
sushiuniABI='[{"inputs":[{"internalType":"address","name":"_factory","type":"address"},{"internalType":"address","name":"_WETH","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"WETH","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"tokenA","type":"address"},{"internalType":"address","name":"tokenB","type":"address"},{"internalType":"uint256","name":"amountADesired","type":"uint256"},{"internalType":"uint256","name":"amountBDesired","type":"uint256"},{"internalType":"uint256","name":"amountAMin","type":"uint256"},{"internalType":"uint256","name":"amountBMin","type":"uint256"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"addLiquidity","outputs":[{"internalType":"uint256","name":"amountA","type":"uint256"},{"internalType":"uint256","name":"amountB","type":"uint256"},{"internalType":"uint256","name":"liquidity","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"amountTokenDesired","type":"uint256"},{"internalType":"uint256","name":"amountTokenMin","type":"uint256"},{"internalType":"uint256","name":"amountETHMin","type":"uint256"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"addLiquidityETH","outputs":[{"internalType":"uint256","name":"amountToken","type":"uint256"},{"internalType":"uint256","name":"amountETH","type":"uint256"},{"internalType":"uint256","name":"liquidity","type":"uint256"}],"stateMutability":"payable","type":"function"},{"inputs":[],"name":"factory","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountOut","type":"uint256"},{"internalType":"uint256","name":"reserveIn","type":"uint256"},{"internalType":"uint256","name":"reserveOut","type":"uint256"}],"name":"getAmountIn","outputs":[{"internalType":"uint256","name":"amountIn","type":"uint256"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountIn","type":"uint256"},{"internalType":"uint256","name":"reserveIn","type":"uint256"},{"internalType":"uint256","name":"reserveOut","type":"uint256"}],"name":"getAmountOut","outputs":[{"internalType":"uint256","name":"amountOut","type":"uint256"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountOut","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"}],"name":"getAmountsIn","outputs":[{"internalType":"uint256[]","name":"amounts","type":"uint256[]"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountIn","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"}],"name":"getAmountsOut","outputs":[{"internalType":"uint256[]","name":"amounts","type":"uint256[]"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountA","type":"uint256"},{"internalType":"uint256","name":"reserveA","type":"uint256"},{"internalType":"uint256","name":"reserveB","type":"uint256"}],"name":"quote","outputs":[{"internalType":"uint256","name":"amountB","type":"uint256"}],"stateMutability":"pure","type":"function"},{"inputs":[{"internalType":"address","name":"tokenA","type":"address"},{"internalType":"address","name":"tokenB","type":"address"},{"internalType":"uint256","name":"liquidity","type":"uint256"},{"internalType":"uint256","name":"amountAMin","type":"uint256"},{"internalType":"uint256","name":"amountBMin","type":"uint256"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"removeLiquidity","outputs":[{"internalType":"uint256","name":"amountA","type":"uint256"},{"internalType":"uint256","name":"amountB","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"liquidity","type":"uint256"},{"internalType":"uint256","name":"amountTokenMin","type":"uint256"},{"internalType":"uint256","name":"amountETHMin","type":"uint256"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"removeLiquidityETH","outputs":[{"internalType":"uint256","name":"amountToken","type":"uint256"},{"internalType":"uint256","name":"amountETH","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"liquidity","type":"uint256"},{"internalType":"uint256","name":"amountTokenMin","type":"uint256"},{"internalType":"uint256","name":"amountETHMin","type":"uint256"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"removeLiquidityETHSupportingFeeOnTransferTokens","outputs":[{"internalType":"uint256","name":"amountETH","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"liquidity","type":"uint256"},{"internalType":"uint256","name":"amountTokenMin","type":"uint256"},{"internalType":"uint256","name":"amountETHMin","type":"uint256"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"},{"internalType":"bool","name":"approveMax","type":"bool"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"removeLiquidityETHWithPermit","outputs":[{"internalType":"uint256","name":"amountToken","type":"uint256"},{"internalType":"uint256","name":"amountETH","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"liquidity","type":"uint256"},{"internalType":"uint256","name":"amountTokenMin","type":"uint256"},{"internalType":"uint256","name":"amountETHMin","type":"uint256"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"},{"internalType":"bool","name":"approveMax","type":"bool"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"removeLiquidityETHWithPermitSupportingFeeOnTransferTokens","outputs":[{"internalType":"uint256","name":"amountETH","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"tokenA","type":"address"},{"internalType":"address","name":"tokenB","type":"address"},{"internalType":"uint256","name":"liquidity","type":"uint256"},{"internalType":"uint256","name":"amountAMin","type":"uint256"},{"internalType":"uint256","name":"amountBMin","type":"uint256"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"},{"internalType":"bool","name":"approveMax","type":"bool"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"removeLiquidityWithPermit","outputs":[{"internalType":"uint256","name":"amountA","type":"uint256"},{"internalType":"uint256","name":"amountB","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountOut","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"swapETHForExactTokens","outputs":[{"internalType":"uint256[]","name":"amounts","type":"uint256[]"}],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountOutMin","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"swapExactETHForTokens","outputs":[{"internalType":"uint256[]","name":"amounts","type":"uint256[]"}],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountOutMin","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"swapExactETHForTokensSupportingFeeOnTransferTokens","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountIn","type":"uint256"},{"internalType":"uint256","name":"amountOutMin","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"swapExactTokensForETH","outputs":[{"internalType":"uint256[]","name":"amounts","type":"uint256[]"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountIn","type":"uint256"},{"internalType":"uint256","name":"amountOutMin","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"swapExactTokensForETHSupportingFeeOnTransferTokens","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountIn","type":"uint256"},{"internalType":"uint256","name":"amountOutMin","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"swapExactTokensForTokens","outputs":[{"internalType":"uint256[]","name":"amounts","type":"uint256[]"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountIn","type":"uint256"},{"internalType":"uint256","name":"amountOutMin","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"swapExactTokensForTokensSupportingFeeOnTransferTokens","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountOut","type":"uint256"},{"internalType":"uint256","name":"amountInMax","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"swapTokensForExactETH","outputs":[{"internalType":"uint256[]","name":"amounts","type":"uint256[]"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amountOut","type":"uint256"},{"internalType":"uint256","name":"amountInMax","type":"uint256"},{"internalType":"address[]","name":"path","type":"address[]"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"deadline","type":"uint256"}],"name":"swapTokensForExactTokens","outputs":[{"internalType":"uint256[]","name":"amounts","type":"uint256[]"}],"stateMutability":"nonpayable","type":"function"},{"stateMutability":"payable","type":"receive"}]'
sushiuniPairABI='[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"spender","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"sender","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount0","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount1","type":"uint256"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"Burn","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"sender","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount0","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount1","type":"uint256"}],"name":"Mint","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"sender","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount0In","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount1In","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount0Out","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"amount1Out","type":"uint256"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"Swap","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint112","name":"reserve0","type":"uint112"},{"indexed":false,"internalType":"uint112","name":"reserve1","type":"uint112"}],"name":"Sync","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"inputs":[],"name":"DOMAIN_SEPARATOR","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"MINIMUM_LIQUIDITY","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"PERMIT_TYPEHASH","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"},{"internalType":"address","name":"","type":"address"}],"name":"allowance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"approve","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"balanceOf","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"to","type":"address"}],"name":"burn","outputs":[{"internalType":"uint256","name":"amount0","type":"uint256"},{"internalType":"uint256","name":"amount1","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"factory","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getReserves","outputs":[{"internalType":"uint112","name":"_reserve0","type":"uint112"},{"internalType":"uint112","name":"_reserve1","type":"uint112"},{"internalType":"uint32","name":"_blockTimestampLast","type":"uint32"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_token0","type":"address"},{"internalType":"address","name":"_token1","type":"address"}],"name":"initialize","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"kLast","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"to","type":"address"}],"name":"mint","outputs":[{"internalType":"uint256","name":"liquidity","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"name","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"nonces","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"},{"internalType":"uint256","name":"deadline","type":"uint256"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"permit","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"price0CumulativeLast","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"price1CumulativeLast","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"to","type":"address"}],"name":"skim","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount0Out","type":"uint256"},{"internalType":"uint256","name":"amount1Out","type":"uint256"},{"internalType":"address","name":"to","type":"address"},{"internalType":"bytes","name":"data","type":"bytes"}],"name":"swap","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"symbol","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"sync","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"token0","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"token1","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"transfer","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"from","type":"address"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"transferFrom","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"}]'
erc20ABI='[{"constant":true,"inputs":[],"name":"name","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"guy","type":"address"},{"name":"wad","type":"uint256"}],"name":"approve","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"totalSupply","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"src","type":"address"},{"name":"dst","type":"address"},{"name":"wad","type":"uint256"}],"name":"transferFrom","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"wad","type":"uint256"}],"name":"withdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"decimals","outputs":[{"name":"","type":"uint8"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[{"name":"","type":"address"}],"name":"balanceOf","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"symbol","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[{"name":"dst","type":"address"},{"name":"wad","type":"uint256"}],"name":"transfer","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":true,"inputs":[{"name":"","type":"address"},{"name":"","type":"address"}],"name":"allowance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"payable":true,"stateMutability":"payable","type":"fallback"},{"anonymous":false,"inputs":[{"indexed":true,"name":"src","type":"address"},{"indexed":true,"name":"guy","type":"address"},{"indexed":false,"name":"wad","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"src","type":"address"},{"indexed":true,"name":"dst","type":"address"},{"indexed":false,"name":"wad","type":"uint256"}],"name":"Transfer","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"dst","type":"address"},{"indexed":false,"name":"wad","type":"uint256"}],"name":"Deposit","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"name":"src","type":"address"},{"indexed":false,"name":"wad","type":"uint256"}],"name":"Withdrawal","type":"event"}]'
bondContractABI='[{"inputs":[{"internalType":"address","name":"_customTreasury","type":"address"},{"internalType":"address","name":"_payoutToken","type":"address"},{"internalType":"address","name":"_principalToken","type":"address"},{"internalType":"address","name":"_olympusTreasury","type":"address"},{"internalType":"address","name":"_subsidyRouter","type":"address"},{"internalType":"address","name":"_initialOwner","type":"address"},{"internalType":"address","name":"_olympusDAO","type":"address"},{"internalType":"uint256[]","name":"_tierCeilings","type":"uint256[]"},{"internalType":"uint256[]","name":"_fees","type":"uint256[]"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"deposit","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"payout","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"expires","type":"uint256"}],"name":"BondCreated","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"internalPrice","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"debtRatio","type":"uint256"}],"name":"BondPriceChanged","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"recipient","type":"address"},{"indexed":false,"internalType":"uint256","name":"payout","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"remaining","type":"uint256"}],"name":"BondRedeemed","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint256","name":"initialBCV","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"newBCV","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"adjustment","type":"uint256"},{"indexed":false,"internalType":"bool","name":"addition","type":"bool"}],"name":"ControlVariableAdjustment","type":"event"},{"inputs":[],"name":"adjustment","outputs":[{"internalType":"bool","name":"add","type":"bool"},{"internalType":"uint256","name":"rate","type":"uint256"},{"internalType":"uint256","name":"target","type":"uint256"},{"internalType":"uint256","name":"buffer","type":"uint256"},{"internalType":"uint256","name":"lastBlock","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"","type":"address"}],"name":"bondInfo","outputs":[{"internalType":"uint256","name":"payout","type":"uint256"},{"internalType":"uint256","name":"vesting","type":"uint256"},{"internalType":"uint256","name":"lastBlock","type":"uint256"},{"internalType":"uint256","name":"truePricePaid","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"bondPrice","outputs":[{"internalType":"uint256","name":"price_","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_olympusTreasury","type":"address"}],"name":"changeOlympusTreasury","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"currentDebt","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"currentOlympusFee","outputs":[{"internalType":"uint256","name":"currentFee_","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"debtDecay","outputs":[{"internalType":"uint256","name":"decay_","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"debtRatio","outputs":[{"internalType":"uint256","name":"debtRatio_","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_amount","type":"uint256"},{"internalType":"uint256","name":"_maxPrice","type":"uint256"},{"internalType":"address","name":"_depositor","type":"address"}],"name":"deposit","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_controlVariable","type":"uint256"},{"internalType":"uint256","name":"_vestingTerm","type":"uint256"},{"internalType":"uint256","name":"_minimumPrice","type":"uint256"},{"internalType":"uint256","name":"_maxPayout","type":"uint256"},{"internalType":"uint256","name":"_maxDebt","type":"uint256"},{"internalType":"uint256","name":"_initialDebt","type":"uint256"}],"name":"initializeBond","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"lastDecay","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"maxPayout","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"paySubsidy","outputs":[{"internalType":"uint256","name":"payoutSinceLastSubsidy_","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_value","type":"uint256"}],"name":"payoutFor","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_depositor","type":"address"}],"name":"pendingPayoutFor","outputs":[{"internalType":"uint256","name":"pendingPayout_","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_depositor","type":"address"}],"name":"percentVestedFor","outputs":[{"internalType":"uint256","name":"percentVested_","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"policy","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_depositor","type":"address"}],"name":"redeem","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bool","name":"_addition","type":"bool"},{"internalType":"uint256","name":"_increment","type":"uint256"},{"internalType":"uint256","name":"_target","type":"uint256"},{"internalType":"uint256","name":"_buffer","type":"uint256"}],"name":"setAdjustment","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"enum CustomBond.PARAMETER","name":"_parameter","type":"uint8"},{"internalType":"uint256","name":"_input","type":"uint256"}],"name":"setBondTerms","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"terms","outputs":[{"internalType":"uint256","name":"controlVariable","type":"uint256"},{"internalType":"uint256","name":"vestingTerm","type":"uint256"},{"internalType":"uint256","name":"minimumPrice","type":"uint256"},{"internalType":"uint256","name":"maxPayout","type":"uint256"},{"internalType":"uint256","name":"maxDebt","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalDebt","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalPayoutGiven","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalPrincipalBonded","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_newOwner","type":"address"}],"name":"transferManagment","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"trueBondPrice","outputs":[{"internalType":"uint256","name":"price_","type":"uint256"}],"stateMutability":"view","type":"function"}]'

ALCX=web3.eth.contract(ALCX_address, abi=erc20ABI)
ALCXETHpool=web3.eth.contract(ALCXETH_pool, abi=sushiuniPairABI)
sushiswapQuoter = web3.eth.contract(address=sushiswapQuoterContract,abi=sushiuniABI)

bondContract_WETHALCX=web3.eth.contract(bondContract_WETHALCX_address,abi=bondContractABI)
bondContract_DAIALCX=web3.eth.contract(bondContract_DAIALCX_address,abi=bondContractABI)
bondContract_CVXALCX=web3.eth.contract(bondContract_CVXALCX_address,abi=bondContractABI)
bondContract_ALCXETHSLP=web3.eth.contract(bondContract_ALCXETHSLP_address,abi=bondContractABI)

In [6]:
############Functions##################
def getAmountsOut_1inch(amountIn,
                        swapPath=["0xdBdb4d16EdA451D0503b854CF79D55697F90c8DF","0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2"]
                        ):
    oneinchQuoteAPI = ('https://api.1inch.exchange/v4.0/1/quote?'+
                       'fromTokenAddress='+swapPath[0]+'&'+
                       'toTokenAddress='+swapPath[1]+'&'+
                        'amount='+str(int(amountIn)))
    oneinchQuoteResponse = requests.get(oneinchQuoteAPI)
    oneinchQuoteResponse = oneinchQuoteResponse.json()
    #print(oneinchQuoteResponse['estimatedGas'])
    swapPath = oneinchQuoteResponse['protocols'][0]
    swapPathDict=[]
    for x in range(len(swapPath)):
        swapPathDict.append(swapPath[x][0])
    swapPathDict
   #print('Doing Work')
    return [int(oneinchQuoteResponse['fromTokenAmount']),int(oneinchQuoteResponse['toTokenAmount']),swapPathDict]

def checkSwap(amountIn,token):
    global path
    amountIn = int(amountIn)
    if amountIn > 0:
        if token=='WETH':
            path=[ALCX_address,tokenDict[token]]
            oneinchquote=getAmountsOut_1inch(amountIn,path)
            amountOut=oneinchquote[1]
            swapPath=oneinchquote[2]
        if token=='DAI':
            path=[ALCX_address,tokenDict[token]]
            oneinchquote=getAmountsOut_1inch(amountIn,path)
            amountOut=oneinchquote[1]
            swapPath=oneinchquote[2]
        if token=='CVX':
            path=[ALCX_address,tokenDict[token]]
            oneinchquote=getAmountsOut_1inch(amountIn,path)
            amountOut=oneinchquote[1]
            swapPath=oneinchquote[2]
        if token=='ALCX-ETH SLP':
            #convert 50% of ALCX to WETH
            path=[ALCX_address,WETH_address]
            oneinchquote=getAmountsOut_1inch(amountIn,path)
            amounts=getAmountsOut_1inch(int(Decimal(amountIn)/2),path)[0:2]
            swapPath=oneinchquote[2]
            reserves=ALCXETHpool.functions.getReserves().call()
            reserveA=reserves[0]
            reserveB=reserves[1]
            LPa=(amounts[1])/reserveA*ALCXETHpool.functions.totalSupply().call()
            LPb=(amounts[0])/reserveB*ALCXETHpool.functions.totalSupply().call()
            amountOut=[min(LPa,LPb),amounts[1]]
    else:
        amountOut=0
        swapPath={}
    #print('spending compute')
    return [amountOut,swapPath]

In [11]:
###########Settings###################
fromAccount='0x0000000000000000000000000000000000000000'
redeemSwapLPBondAddress="0xbaDb0710c62454307a1a9c50aBc9967601A78A81"
#Set Max Slippages
swapSlippage=0.005 #to match sushiswap set = 0.005 (0.5%), this gets handled on 1inch api
addLiquiditySlippage=0.005 #to match zapper default = 0.02 (2%)
deadlineMinutes=5 #5 minutes before tx is invalid, immediately fails stale orders
######################################
debugMode=False

fromAccount_web3=web3.toChecksumAddress(fromAccount)
redeemSwapLPBondContract=web3.eth.contract(Web3.toChecksumAddress(redeemSwapLPBondAddress), abi=reedeemSwapLPBondABI)

olympusFee = bondContract_WETHALCX.functions.currentOlympusFee().call()

pendingBond_WETHALCX=bondContract_WETHALCX.functions.bondInfo(fromAccount_web3).call()[0]
claimableBond_WETHALCX=bondContract_WETHALCX.functions.pendingPayoutFor(fromAccount_web3).call()
maxInput_WETHALCX=(bondContract_WETHALCX.functions.maxPayout().call()*bondContract_WETHALCX.functions.bondPrice().call()/10**7/(1-olympusFee/10**6))
maxPayout_WETHALCX=bondContract_WETHALCX.functions.maxPayout().call()

pendingBond_DAIALCX=bondContract_DAIALCX.functions.bondInfo(fromAccount_web3).call()[0]
claimableBond_DAIALCX=bondContract_DAIALCX.functions.pendingPayoutFor(fromAccount_web3).call()
maxInput_DAIALCX=(bondContract_DAIALCX.functions.maxPayout().call()*bondContract_DAIALCX.functions.bondPrice().call()/10**7/(1-olympusFee/10**6))
maxPayout_DAIALCX=bondContract_DAIALCX.functions.maxPayout().call()

pendingBond_CVXALCX=bondContract_CVXALCX.functions.bondInfo(fromAccount_web3).call()[0]
claimableBond_CVXALCX=bondContract_CVXALCX.functions.pendingPayoutFor(fromAccount_web3).call()
maxInput_CVXALCX=(bondContract_CVXALCX.functions.maxPayout().call()*bondContract_CVXALCX.functions.bondPrice().call()/10**7/(1-olympusFee/10**6))
maxPayout_CVXALCX=bondContract_CVXALCX.functions.maxPayout().call()

pendingBond_ALCXETHSLP=bondContract_ALCXETHSLP.functions.bondInfo(fromAccount_web3).call()[0]
claimableBond_ALCXETHSLP=bondContract_ALCXETHSLP.functions.pendingPayoutFor(fromAccount_web3).call()
maxInput_ALCXETHSLP=(bondContract_ALCXETHSLP.functions.maxPayout().call()*bondContract_ALCXETHSLP.functions.bondPrice().call()/10**7/(1-olympusFee/10**6))
maxPayout_ALCXETHSLP=bondContract_ALCXETHSLP.functions.maxPayout().call()

walletBalance=ALCX.functions.balanceOf(fromAccount_web3).call()

useWalletBalance = widgets.Checkbox(description="Use Wallet Balance"+" ("+str(round(walletBalance/10**18,5))+")")
walletSlider =widgets.FloatSlider(value=walletBalance/10**18,
    min=0,
    max=walletBalance/10**18,
    step=0.00001,
    description='',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
)


def generateMulticall(b):
    #redeems
    calldata=[]
    deadline=(math.floor(time.time())+60*deadlineMinutes) #5 minmutes to exeucte
    
    redeems=[]
    if WETHadd>0:
        redeems.append(bondContract_WETHALCX.address)
    if DAIadd>0:
        redeems.append(bondContract_DAIALCX.address)
    if CVXadd>0:
        redeems.append(bondContract_CVXALCX.address)
    if ALCXETHadd>0:
        redeems.append(bondContract_ALCXETHSLP.address)
    
    slippages=[]
    if symbol == 'ALCX-ETH SLP':
        slippages.append(int(amountIn*10**18*(1-addLiquiditySlippage)))
        slippages.append(int(amountOut*10**18*(1-addLiquiditySlippage)))
    else:
        slippages.append(0)
        slippages.append(0)
    slippages.append(deadline)
    slippages.append(trueBondPrice+5000)
    
    #GENERATE 1 INCH CALLDATA                    
    oneinchOutputToken = WETH_address if symbol == "ALCX-ETH SLP" else tokenDict[symbol] 
    oneinchSwapAPI = ('https://api.1inch.exchange/v4.0/1/swap?'+
        'fromTokenAddress='+ALCX_address+'&'+
        'toTokenAddress='+oneinchOutputToken+'&'+
        'fromAddress='+redeemSwapLPBondAddress+'&'+
        'amount='+str(int(amountIn))+'&'+
        'slippage='+str(swapSlippage*100)+'&'+
        'disableEstimate=true')
    oneinchSwapResponse = requests.get(oneinchSwapAPI)
    oneinchSwapResponse = oneinchSwapResponse.json()
    oneinchcalldata = oneinchSwapResponse['tx']['data']
                
    if amountIn>0:
        if symbol == 'ALCX-ETH SLP':           
            structIn={'redeems': redeems, 
              'slippages': slippages,
              'amountIn': int(Decimal(amountIn)*2),
              'tokenIn': path[0],
              'tokenOut': path[1],
              'swapPayload': web3.toBytes(hexstr=oneinchcalldata),
              'LP': True,
              'poolAddress': ALCXETH_pool,
              'bondContractAddress': bondContractAddress
             }
            rSLBCdetails=redeemSwapLPBondContract.functions.redeemSwapLPBond(structIn).buildTransaction({'gas':0})
            decoded_data=redeemSwapLPBondContract.decode_function_input(rSLBCdetails['data'])
            rSLBCDict={'type': 'rsLBC', 'to': rSLBCdetails['to'],'data': rSLBCdetails['data'], 'decode': decoded_data}
            calldata.append(rSLBCDict)
        else:
            structIn={'redeems': redeems, 
              'slippages': slippages,
              'amountIn': int(amountIn),
              'tokenIn': path[0],
              'tokenOut': path[1],
              'swapPayload': web3.toBytes(hexstr=oneinchcalldata),
              'LP': False,
              'poolAddress': "0x0000000000000000000000000000000000000000",
              'bondContractAddress': bondContractAddress
             }
            rSLBCdetails=redeemSwapLPBondContract.functions.redeemSwapLPBond(structIn).buildTransaction({'gas':0})
            decoded_data=redeemSwapLPBondContract.decode_function_input(rSLBCdetails['data'])
            rSLBCDict={'type': 'rsLBC', 'to': rSLBCdetails['to'],'data': rSLBCdetails['data'], 'decode': decoded_data}
            calldata.append(rSLBCDict)
    #output call data
    calls=calldata

    decode_encoded = ''
    multicallInputs=[]
    decodedInputs=[]
    debugData.value=''
    slippages_str = []
    for n in slippages:
        slippages_str.append(str(n))
    redeems_str = []
    for n in redeems:
        redeems_str.append(str(n))

    outputData=("["+
    str(redeems_str)+","+
    str(slippages_str)+","+
    '"'+str(structIn['amountIn'])+'"'+","+
    '"'+str(structIn['tokenIn'])+'"'+","+
    '"'+str(structIn['tokenOut'])+'"'+","+
    '"'+str(web3.toHex(structIn['swapPayload']))+'"'+","+
        str(structIn['LP']).lower()+","+
    '"'+str(structIn['poolAddress'])+'"'+","+
    '"'+str(structIn['bondContractAddress'])+'"'+"]")
    outputData=re.sub("'", '"', outputData)

    for call in calls:
        multicallInputs.append([call['to'],call['data']])
        debugData.value=debugData.value+str(decode_encoded)+'\n\n'
        #decodedInputs.append(call['decode'])    
    #debugData.value=debugData.value+str(oneinchSwapAPI)+'\n\n'
    #debugData.value=debugData.value+str(oneinchSwapResponse)
    #multicallData.value=str(multicallInputs).replace("'",'"')
    multicallData.value=outputData

    #debugData.value=str(decodedInputs)
    maxInput=float(re.sub("[^0-9,.]", "", poolDetails_maxInput.value))
    currentInput=amountSLP if symbol=='ALCX-ETH SLP' else amountOut
    if currentInput/10**18 > maxInput:
        multicallData.value=debugData.value=str('You are attempting to deposit more than the max input for the current bond, adjust wallet balance slider or remove a redeem to continue.')
    #copy multicall data to clipboard

    pc.copy(multicallData.value)
    

redeemBond_WETHALCX = widgets.Checkbox(description="WETH/ALCX"+" ("+str(round(claimableBond_WETHALCX/10**18,5))+")")
redeemBond_DAIALCX= widgets.Checkbox(description="DAI/ALCX"+" ("+str(round(claimableBond_DAIALCX/10**18,5))+")")
redeemBond_CVXALCX= widgets.Checkbox(description="CVX/ALCX"+" ("+str(round(claimableBond_CVXALCX/10**18,5))+")")
redeemBond_ALCXETHSLP= widgets.Checkbox(description="ALCX-ETH SLP"+" ("+str(round(claimableBond_ALCXETHSLP/10**18,5))+")")

totalRedeemOut=widgets.Label()
totalSupplyOut=widgets.Label()
totalTransferIn=widgets.Label()
swapPathLabel=widgets.Label()
swapPathOut=widgets.Textarea(layout=widgets.Layout(height="60px",width="auto"))

poolDetails_maxInput=widgets.Label()
poolDetails_maxPayout=widgets.Label()
poolDetails_yourPayout=widgets.Label()
poolDetails_ROI=widgets.Label()
#initialize
poolDetails_maxInput.value='Max Input: '+str(maxInput_WETHALCX/10**18)+' WETH'
poolDetails_yourPayout.value='You Will Get: '+str(0)+' ALCX'
poolDetails_maxPayout.value='Max Payout: '+str(maxPayout_WETHALCX/10**18)+' ALCX'
poolDetails_ROI.value='ROI: '

multicallData=widgets.Textarea(layout=widgets.Layout(height="200px",width="auto"))
debugData=widgets.Textarea(layout=widgets.Layout(height="200px",width="auto"))

depositBond_Dropdown = widgets.Dropdown(options=["WETH/ALCX","DAI/ALCX","CVX/ALCX","ALCX-ETH SLP"],description="")

#set inital values
WETHadd=DAIadd=CVXadd=ALCXETHadd=walletAdd=amountIn=amountOut=0
bondPrice=bondContract_WETHALCX.functions.bondPrice().call()
symbol='WETH'
bondContractAddress=bondContract_WETHALCX.address
trueBondPrice=bondContract_WETHALCX.functions.trueBondPrice().call()
walletAdd=walletBalance
walletCheck=False



def window_open_button(url):
    with out:
        display(Javascript(f'window.open("{url.tooltip}");'))

        
def calculateCycle(b):
    global WETHadd,DAIadd,CVXadd,ALCXETHadd,walletAdd,symbol,amountIn,amountOut,amountSLP,bondPrice,trueBondPrice,path,swapPath,bondContractAddress,walletIn,walletCheck
    global fromAccount,walletBalance,pendingBond_WETHALCX,claimableBond_WETHALCX,pendingBond_DAIALCX,claimableBond_DAIALCX,pendingBond_CVXALCX,claimableBond_CVXALCX,pendingBond_ALCXETHSLP,claimableBond_ALCXETHSLP
    if type(b['owner']).__name__ == 'Text' and b['new']=={}:
        if(re.search('.eth',enterAddress.value)):
            fromAccount = ns.address(enterAddress.value)
            enterAddress.value = fromAccount
        if(web3.isAddress(enterAddress.value)):
            currentBlock_label.value = '[Current block: '+str(web3.eth.blockNumber)+']'
            fromAccount=enterAddress.value
            fromAccount_web3=web3.toChecksumAddress(fromAccount)
            enterAddress.value = fromAccount
            pendingBond_WETHALCX=bondContract_WETHALCX.functions.bondInfo(fromAccount_web3).call()[0]
            claimableBond_WETHALCX=bondContract_WETHALCX.functions.pendingPayoutFor(fromAccount_web3).call()
            pendingBond_DAIALCX=bondContract_DAIALCX.functions.bondInfo(fromAccount_web3).call()[0]
            claimableBond_DAIALCX=bondContract_DAIALCX.functions.pendingPayoutFor(fromAccount_web3).call()
            pendingBond_CVXALCX=bondContract_CVXALCX.functions.bondInfo(fromAccount_web3).call()[0]
            claimableBond_CVXALCX=bondContract_CVXALCX.functions.pendingPayoutFor(fromAccount_web3).call()
            pendingBond_ALCXETHSLP=bondContract_ALCXETHSLP.functions.bondInfo(fromAccount_web3).call()[0]
            claimableBond_ALCXETHSLP=bondContract_ALCXETHSLP.functions.pendingPayoutFor(fromAccount_web3).call()
            walletBalance=ALCX.functions.balanceOf(fromAccount_web3).call()
            
            redeemBond_WETHALCX.description = "WETH/ALCX"+" ("+str(round(claimableBond_WETHALCX/10**18,5))+")"
            redeemBond_DAIALCX.description = "DAI/ALCX"+" ("+str(round(claimableBond_DAIALCX/10**18,5))+")"
            redeemBond_CVXALCX.description = "CVX/ALCX"+" ("+str(round(claimableBond_CVXALCX/10**18,5))+")"
            redeemBond_ALCXETHSLP.description = "ALCX-ETH SLP"+" ("+str(round(claimableBond_ALCXETHSLP/10**18,5))+")"
            useWalletBalance.description = "Use Wallet Balance"+" ("+str(round(walletBalance/10**18,5))+")"
            
            redeemBond_WETHALCX.value = False
            redeemBond_DAIALCX.value = False
            redeemBond_CVXALCX.value =False
            redeemBond_ALCXETHSLP.value = False
            useWalletBalance.value = False
            
            walletSlider.value = walletBalance/10**18
            walletSlider.max = walletBalance/10**18
    
    if type(b['owner']).__name__ == 'FloatSlider' and b['new']=={}:
        if(walletCheck==True):
            walletAdd=min(walletBalance,walletSlider.value*10**18)
        #print(walletAdd)
    if type(b['owner']).__name__ == 'Checkbox' and type(b['new'])==bool:
        if re.search('Wallet',b['owner'].description):
            walletAdd= walletAdd if b['new'] else 0.0
            walletCheck= True if b['new'] else False
            if(walletCheck==True):
                walletAdd=min(walletBalance,walletSlider.value*10**18)
        if re.search('WETH',b['owner'].description):
            WETHadd = claimableBond_WETHALCX if b['new'] else 0.0
        if re.search('DAI',b['owner'].description):
            DAIadd = claimableBond_DAIALCX if b['new'] else 0.0
        if re.search('CVX',b['owner'].description):
            CVXadd = claimableBond_CVXALCX if b['new'] else 0.0
        if re.search('ALCX-ETH',b['owner'].description):
            ALCXETHadd = claimableBond_ALCXETHSLP if b['new'] is True else 0.0
        amountIn=WETHadd+DAIadd+CVXadd+ALCXETHadd+walletAdd
        swapQuery=checkSwap(amountIn,symbol)
        amountOut=swapQuery[0]
        swapPath=swapQuery[1]
        
    if type(b['owner']).__name__ == 'Dropdown' and b['new']=={}:
        symbol = re.sub('/ALCX','',depositBond_Dropdown.value)
        swapQuery=checkSwap(amountIn,symbol)
        amountOut=swapQuery[0]
        swapPath=swapQuery[1]
        if symbol=='WETH':
            bondContractAddress=bondContract_WETHALCX.address
            poolDetails_maxInput.value='Max Input: '+str(maxInput_WETHALCX/10**18)+' '+symbol
            poolDetails_maxPayout.value='Max Payout: '+str(maxPayout_WETHALCX/10**18)+' '+'ALCX'
            bondPrice=bondContract_WETHALCX.functions.bondPrice().call()
            trueBondPrice=bondContract_WETHALCX.functions.trueBondPrice().call()
        if symbol=='DAI':
            bondContractAddress=bondContract_DAIALCX.address
            poolDetails_maxInput.value='Max Input: '+str(maxInput_DAIALCX/10**18)+' '+symbol
            poolDetails_maxPayout.value='Max Payout: '+str(maxPayout_DAIALCX/10**18)+' '+'ALCX'
            bondPrice=bondContract_DAIALCX.functions.bondPrice().call()
            trueBondPrice=bondContract_DAIALCX.functions.trueBondPrice().call()
        if symbol=='CVX':
            bondContractAddress=bondContract_CVXALCX.address
            trueBondPrice=bondContract_CVXALCX.functions.trueBondPrice().call()
            poolDetails_maxInput.value='Max Input: '+str(maxInput_CVXALCX/10**18)+' '+symbol
            poolDetails_maxPayout.value='Max Payout: '+str(maxPayout_CVXALCX/10**18)+' '+'ALCX'
            bondPrice=bondContract_CVXALCX.functions.bondPrice().call()
            trueBondPrice=bondContract_CVXALCX.functions.trueBondPrice().call()
        if symbol=='ALCX-ETH SLP':
            bondContractAddress=bondContract_ALCXETHSLP.address
            trueBondPrice=bondContract_ALCXETHSLP.functions.trueBondPrice().call()
            poolDetails_maxInput.value='Max Input: '+str(maxInput_ALCXETHSLP/10**18)+' '+symbol
            poolDetails_maxPayout.value='Max Payout: '+str(maxPayout_ALCXETHSLP/10**18)+' '+'ALCX'
            bondPrice=bondContract_ALCXETHSLP.functions.bondPrice().call()
            trueBondPrice=bondContract_ALCXETHSLP.functions.trueBondPrice().call()

    if WETHadd+DAIadd+CVXadd+ALCXETHadd+walletAdd==0:
        totalTransferIn.value=""
        totalRedeemOut.value=""
        totalSupplyOut.value=""
        swapPathLabel.value=""
        swapPathOut.value=""
        poolDetails_yourPayout.value='You Will Get: '+str(0)+' ALCX'
        poolDetails_ROI.value='ROI: '
    else:
        if(web3.isAddress(enterAddress.value)):
            if b['new']=={}:
                if symbol=='ALCX-ETH SLP':
                    if(walletCheck==True):
                        walletAdd=min(walletAdd,walletSlider.value*10**18)
                    amountIn=WETHadd+DAIadd+CVXadd+ALCXETHadd+walletAdd
                    swapQuery=checkSwap(amountIn,symbol)
                    amountOut=swapQuery[0]
                    swapPath=swapQuery[1]
                    totalTransferIn.value="Transferring "+str(round(amountIn/10**18,5))+" ALCX into Smart Contract"
                    amountIn=amountIn/2
                    amountSLP=amountOut[0]
                    amountOut=amountOut[1]
                    totalRedeemOut.value="Swapping "+str(round(amountIn/10**18,5))+" ALCX for "+str(round(amountOut/10**18,5))+" WETH"
                    totalSupplyOut.value="Supplying "+str(round(amountOut/10**18,5))+" WETH and "+str(round(amountIn/10**18,5))+" ALCX for "+str(round(amountSLP/10**18,5))+" ALCX-ETH SLP"
                    swapPathLabel.value="Swap Path:"
                    swapPathOut.value=str(swapPath)
                    payout=amountSLP/bondPrice/10**11*(1-olympusFee/10**6)
                    poolDetails_ROI.value='ROI: '+str(round(100-(amountIn*2/10**18/payout)*100,2))+'%'

                else:
                    if(walletCheck==True):
                        walletAdd=min(walletAdd,walletSlider.value*10**18)
                    amountIn=WETHadd+DAIadd+CVXadd+ALCXETHadd+walletAdd
                    swapQuery=checkSwap(amountIn,symbol)
                    amountOut=swapQuery[0]
                    swapPath=swapQuery[1]
                    totalTransferIn.value="Transferring "+str(round(amountIn/10**18,5))+" ALCX into Smart Contract"
                    totalRedeemOut.value="Swapping "+str(round(amountIn/10**18,5))+" ALCX for "+str(round(amountOut/10**18,5))+" "+symbol
                    totalSupplyOut.value=""
                    swapPathLabel.value="Swap Path:"
                    swapPathOut.value=str(swapPath)
                    payout=amountOut/bondPrice/10**11*(1-olympusFee/10**6)
                    poolDetails_ROI.value='ROI: '+str(round(100-(amountIn/10**18/payout)*100,2))+'%'

                poolDetails_yourPayout.value='You Will Get: '+str(payout)+' ALCX'


enterAddress=widgets.Text(value=fromAccount_web3,description='Address: ', layout=Layout(width='425px'))
generateMC = widgets.Button(description='Generate MultiCall Data',disabled=False,layout=Layout(width='200px'))
generateMC.on_click(generateMulticall)
            
enterAddress.observe(calculateCycle)
useWalletBalance.observe(calculateCycle)
walletSlider.observe(calculateCycle)
redeemBond_WETHALCX.observe(calculateCycle)
redeemBond_DAIALCX.observe(calculateCycle)
redeemBond_CVXALCX.observe(calculateCycle)
redeemBond_ALCXETHSLP.observe(calculateCycle)
depositBond_Dropdown.observe(calculateCycle)

currentBlock_label=widgets.Label('[Current block: '+str(web3.eth.blockNumber)+']')

out = widgets.Output()

etherscan = widgets.Button(description="Open Etherscan", tooltip=('https://etherscan.io/address/'+redeemSwapLPBondAddress+'#writeContract'))
etherscan.on_click(window_open_button)



ui = widgets.VBox([  
                     currentBlock_label,
                     enterAddress,
                     widgets.Label(value='Select Assets to Claim:'),
                     widgets.HBox([useWalletBalance,walletSlider]),
                     widgets.HBox([redeemBond_WETHALCX, redeemBond_DAIALCX]), 
                     widgets.HBox([redeemBond_CVXALCX, redeemBond_ALCXETHSLP]),
                     widgets.VBox([totalTransferIn,totalRedeemOut,totalSupplyOut]),
                     swapPathLabel,
                     swapPathOut,
                     widgets.Label(value='Select Pool to Bond:'),
                     depositBond_Dropdown,
                     poolDetails_maxInput,
                     poolDetails_yourPayout,
                     poolDetails_ROI,
                     poolDetails_maxPayout,
                     widgets.HBox([generateMC,etherscan]),
                     multicallData,
                     widgets.Label('Debug:') if debugMode else widgets.Label(), #COMMENT OUT
                     debugData if debugMode else widgets.Label() #COMMENT OUT
                  ])
ui
